## 예술의전당 콘서트홀 가격 모델

### 1. 라이브러리

In [163]:
# base
import pandas as pd
import numpy as np
import re

# files
from glob import glob

# visualization
import matplotlib.pyplot as plt
import koreanize_matplotlib
import plotly.express as px
import seaborn as sns

In [164]:
# settings
%matplotlib inline
pd.set_option("display.max_columns", 100)
pd.set_option('display.max_rows', 100)


### 2. 데이터셋 로드
df_book : 예매데이터  
df_seat : 좌석데이터 (좌표)

In [165]:
def load_file (file_name, file_num=0):
    files = glob(file_name)
    file = files[file_num]   
    df = pd.read_csv(files[file_num])

    print(f"Found.. {len(files)} file(s) : {files}")    
    print(f"Reading.. {files[file_num]}")
    print(f"dataframe.shape : {df.shape}")
    
    return df

#### 2-1) df_book : 예매데이터

In [278]:
# df_book : 공연예매정보
df_book = load_file('2023-08-15*.csv')

# 장소(콘서트홀), 장르(클래식)으로 특정
df_book = df_book[(df_book['place']=='콘서트홀') & (df_book['genre']=='클래식')]
print(f"df_book.shape : {df_book.shape}")

Found.. 1 file(s) : ['2023-08-15 가공 데이터.csv']
Reading.. 2023-08-15 가공 데이터.csv
dataframe.shape : (1920868, 24)
df_book.shape : (236966, 24)


변수 타입, 파생변수 생성

In [279]:
# pd.to_datetime
df_book['공연일시'] = pd.to_datetime(df_book['play_date'] + " " + df_book['play_st_time'])
df_book['예매일시'] = pd.to_datetime(df_book['tran_date'] + " " + df_book['tran_time'])
df_book = df_book.drop(['place', 'genre', 'play_date', 'play_st_time', 'tran_date', 'tran_time'], axis=1)

같은 공연, 좌석에서 중복된 데이터 삭제

In [280]:
# 중복제거 : 티켓별로 최종구매 건만 남기기
print(f"로우 데이터 : {df_book.shape[0]}")

# step1 : 취소되지 않은 경우(ticket_cancel==0)으로 한정해서 보더라도, 같은 공연 같은 좌석에 대한 예매 기록이 중복되어 있음 => 전산상의 오류?
temp = df_book.loc[df_book['ticket_cancel'] == 0]
print(f"step1 (취소표 제거) : {temp[['공연일시', 'seat', 'ticket_cancel']].value_counts().sum()}")

# step2 : step1의 결과가 전산상의 오류라고 가정하고, 예매일시 기준으로 제일 나중 기록만 남긴다.
temp = temp.groupby(['공연일시', 'seat']).agg({'예매일시':'max'})
temp_merged = df_book.merge(temp.reset_index(), how='right', on=['공연일시', 'seat', '예매일시'])
print(f"step2 (예매일시별 최종구매건만) : {temp_merged[['공연일시', 'seat', 'ticket_cancel']].value_counts().count()}")

# step3 : 예매일시까지 동일한 기록이 1개 있음 => 해당 데이터만 삭제
# (75569) 2020-06-03 19:30:00  1층 B블록22열 2
target_idx = temp_merged.loc[(temp_merged['공연일시']=='2020-06-03 19:30:00') 
                             & (temp_merged['seat']=='1층 B블록22열 2')].reset_index()['index'][1]
temp_merged = temp_merged.drop(target_idx)
temp_merged = temp_merged.reset_index(drop=True)
print(f"step3 (75569th 행 제거) : {temp_merged[['공연일시', 'seat', 'ticket_cancel']].value_counts().count()}")

df_book_2 = temp_merged.copy()

로우 데이터 : 236966
step1 (취소표 제거) : 186612
step2 (예매일시별 최종구매건만) : 180967
step3 (75569th 행 제거) : 180966


#### 2-2) df_seat : 좌석데이터

In [281]:
# df_seat : 좌석정보
df_seat = load_file('seat*.csv')
df_seat.columns = ['층', '블록', '열', '넘버', 'seat', 'X', 'Y', 'Z', '대칭점']

Found.. 1 file(s) : ['seat_coordinate_info (symmetry point).csv']
Reading.. seat_coordinate_info (symmetry point).csv
dataframe.shape : (2505, 9)


### 3. df_book에 구매되지 않은 좌석도 채워넣기

In [ ]:
# 공연 - 좌석별 칸 만들어놓기
# NaN의 경우 value_counts에서 카운트되지 않는다 > 0으로 채워서 NaN인 경우도 볼 수 있도록 한다.
temp = df_book[['공연일시', 'performance_code', 'pre_open_date', 'open_date', 'running_time', 'intermission']].fillna(0).value_counts().reset_index()
temp = temp.drop(0, axis=1)

# 공연별(162) - 좌석별(2505) merge
df_temp_merged = temp.merge(df_seat, how='cross')
print(f"공연별({temp.shape[0]}) * 좌석별({df_seat.shape[0]}) = {df_temp_merged.shape[0]}")
print(temp.shape[0] * df_seat.shape[0] == df_temp_merged.shape[0])

In [308]:
# 중복되는 컬럼을 삭제
duplicated_cols = list(set(df_temp_merged.columns).intersection(set(df_book_2.columns)) - {'seat', '공연일시'})
df_book_2 = df_book_2.drop(duplicated_cols, axis=1)

# df_book_2는 중복구매만 제거한 데이터다. 티켓이 판매된 경우만 특정한 것이기 떄문에 '판매여부'컬럼에 모두 True로 표기한다.
df_book_2['판매여부'] = True

# df_temp_merged는 공연별로 2505개의 좌석이 들어가게 만들어놓은 틀이다. df_book_2와 합치면서 df_book_2에는 없는 데이터라면 '판매여부'를 False로 채운다.
df_merged = df_book_2.merge(df_temp_merged, how='right', on=['seat', '공연일시'])
df_merged['판매여부'] = df_merged['판매여부'].fillna(False)
df_merged

,age,gender,membership_type_1,membership_type_2,membership_type_3,membership_type_4,membership_type_5,membership_type_6,seat,price,ticket_cancel,discount_type,member_yn,공연일시,예매일시,판매여부,performance_code,pre_open_date,open_date,running_time,intermission,층,블록,열,넘버,X,Y,Z,대칭점
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1층 A블록1열 1,NaN,NaN,NaN,NaN,2021-03-31 19:30:00,NaT,False,1725,2021-02-13,2021-02-20,120,15,1층,A블록,1,1,1451,542,-93,1층 E블록1열 9
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1층 A블록1열 2,NaN,NaN,NaN,NaN,2021-03-31 19:30:00,NaT,False,1725,2021-02-13,2021-02-20,120,15,1층,A블록,1,2,1406,555,-93,1층 E블록1열 8
2,40.0,F,무료,그린,NaN,NaN,NaN,NaN,1층 A블록1열 3,96000.0,0.0,그린회원 할인20%,Y,2021-03-31 19:30:00,2021-03-18 17:39:00,True,1725,2021-02-13,2021-02-20,120,15,1층,A블록,1,3,1361,568,-93,1층 E블록1열 7
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1층 A블록1열 4,NaN,NaN,NaN,NaN,2021-03-31 19:30:00,NaT,False,1725,2021-02-13,2021-02-20,120,15,1층,A블록,1,4,1315,580,-93,1층 E블록1열 6
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1층 A블록1열 5,NaN,NaN,NaN,NaN,2021-03-31 19:30:00,NaT,False,1725,2021-02-13,2021-02-20,120,15,1층,A블록,1,5,1270,591,-93,1층 E블록1열 5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
405805,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2층 BOX6 2,NaN,NaN,NaN,NaN,2019-12-25 20:00:00,NaT,False,2932,2019-11-29,2019-11-30,100,15,2층,BOX6,없음,2,-1432,505,383,2층 BOX1 1
405806,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2층 BOX6 3,NaN,NaN,NaN,NaN,2019-12-25 20:00:00,NaT,False,2932,2019-11-29,2019-11-30,100,15,2층,BOX6,없음,3,-1500,565,428,2층 BOX1 4
405807,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2층 BOX6 4,NaN,NaN,NaN,NaN,2019-12-25 20:00:00,NaT,False,2932,2019-11-29,2019-11-30,100,15,2층,BOX6,없음,4,-1454,617,428,2층 BOX1 3
405808,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2층 BOX6 5,NaN,NaN,NaN,NaN,2019-12-25 20:00:00,NaT,False,2932,2019-11-29,2019-11-30,100,15,2층,BOX6,없음,5,-1522,677,483,2층 BOX1 6


### 4. 원가격추정-1 : 판매된 좌석의 할인 전 금액 추정하기

discount_type 에서 '할인율' 혹은 '할인액'을 구해서 'price'에서 역산한다.

In [309]:
# discount_type에 적용된 할인의 종류가 다양함.
df_merged['discount_type'].value_counts()

초대권                93749
일반                 22264
기획사판매              21029
골드회원 할인10%          6634
그린회원 할인5%           3307
                   ...  
골드회원(법인A) 할인30%        1
기획사할인50%               1
골드회원 할인10%_            1
특판B 30%                1
그린회원 할인30%             1
Name: discount_type, Length: 213, dtype: int64

In [314]:
# 할인율 계산 > % 앞의 숫자만 추출
df_merged['할인율'] = df_merged['discount_type'].str.extract('(\d+)%')
df_merged['할인율'] = df_merged['할인율'].fillna(0).astype(int)
df_merged['할인율'] = df_merged['할인율'] / 100
print(df_merged['할인율'].value_counts().sort_index())

# NaN인 경우 (티켓이 판매되지 않은 경우) 채워넣기
df_merged[['할인율', 'price']] = df_merged[['할인율', 'price']].fillna(0)

0.00    368076
0.05      9230
0.10     10125
0.12       640
0.15       974
0.20      4371
0.25       372
0.30      5351
0.40      3597
0.50      3074
Name: 할인율, dtype: int64


In [315]:
# discount_type에 할인율이 명시되어 있지 않은 경우 > 대부분 초대권, 기획사판매, 예매권
df_merged.loc[df_merged['할인율']==0, 'discount_type'].value_counts()

초대권                  93749
일반                   22264
기획사판매                21029
공연진행석                 2406
홍보진행                  1131
당일할인티켓                 981
기획사할인                  695
차액                     339
중앙일보 JTBC              200
당일할인티켓_                116
중앙일보 JTBC 초대권          100
공연예매권                   88
기획사                     18
하비에르 국제학교 학부모, 직원       13
수험생 할인(동반1인)            10
하비에르 국제학교 재학생            9
싹틔우미 할인                  3
Name: discount_type, dtype: int64

In [316]:
# '할인전금액' 구하기 : 'price(판매금액)'에 '할인율'을 역산
df_merged['할인전금액'] = df_merged['price'] / (1 - df_merged['할인율'])
df_merged['할인전금액'] = df_merged['할인전금액'].astype(int)

df_merged[['discount_type', 'price', '할인율', '할인전금액']]

,discount_type,price,할인율,할인전금액
0,NaN,0.0,0.0,0
1,NaN,0.0,0.0,0
2,그린회원 할인20%,96000.0,0.2,120000
3,NaN,0.0,0.0,0
4,NaN,0.0,0.0,0
...,...,...,...,...
405805,NaN,0.0,0.0,0
405806,NaN,0.0,0.0,0
405807,NaN,0.0,0.0,0
405808,NaN,0.0,0.0,0


### 5. 원가격추정-2 : 판매되지 않은 좌석의 티켓값 구하기
주어진 데이터(df_book)에는 판매된 좌석의 티켓금액만 명시되어 있음.
판매되지 않은 좌석은 오픈되었는지, 얼마에 판매되고 있었는지 나와있지 않음.

1) df_book, df_seat을 merge해서 공연별로 2505개의 좌석을 갖도록 만들어준다.
2) 컬럼을 추가해서 판매여부를 명시한다.
3) 공연별로 knn모델을 이용해서 적절한 n-neighbors값을 찾는다. (valid데이터셋으로 확인)
4) 공연별로 찾은 n-neighbors값을 적용해서 판매금액을 추정한다.

In [285]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error